<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import json
import datetime
from math import log
import pandas as pd

In [2]:
# input file name: ./input/stockPrices_raw.json
# output file name: ./input/stockReturns.json
# json structure: crawl daily price data from yahoo finance
#          term (short/mid/long)
#         /         |         \
#   ticker A   ticker B   ticker C
#      /   \      /   \      /   \
#  date1 date2 date1 date2 date1 date2
#
# Note: short return: adjClose/open - 1
#       mid return: adjClose/adjClose(7 days ago) - 1
#       long return: adjClose/adjClose(28 days ago) - 1

In [3]:
def calc_mid_long_return(ticker, date, delta, priceSet):
    baseDate = datetime.datetime.strptime(date, "%Y-%m-%d")
    prevDate = (baseDate - datetime.timedelta(days=delta)).strftime("%Y-%m-%d")
    nextDate = (baseDate + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    if delta == 1:
        wkday = baseDate.weekday()
        if wkday == 0: # Monday
            prevDate = (baseDate - datetime.timedelta(days=3)).strftime("%Y-%m-%d")
        elif wkday == 4: # Friday
            nextDate = (baseDate + datetime.timedelta(days=3)).strftime("%Y-%m-%d")
        elif wkday == 5: # Saturday
            prevDate = (baseDate - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
            nextDate = (baseDate + datetime.timedelta(days=2)).strftime("%Y-%m-%d")
        elif wkday == 6: # Sunday
            prevDate = (baseDate - datetime.timedelta(days=2)).strftime("%Y-%m-%d")
            nextDate = (baseDate + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    try:
        
        return_self = (priceSet[ticker]['adjClose'][date] - priceSet[ticker]['adjClose'][prevDate]) / priceSet[ticker]['adjClose'][prevDate]
        return_sp500 = (priceSet['^GSPC']['adjClose'][date] - priceSet['^GSPC']['adjClose'][prevDate]) / priceSet['^GSPC']['adjClose'][prevDate]
        return_self_per = round(return_self, 4) * 100
        return_sp500_per = round(return_sp500, 4) * 100
        
#         else:
#             return_self = (priceSet[ticker]['adjClose'][date] - priceSet[ticker]['open'][prevDate]) / priceSet[ticker]['open'][prevDate]
#             return_sp500 = (priceSet['^GSPC']['adjClose'][date] - priceSet['^GSPC']['open'][prevDate]) / priceSet['^GSPC']['open'][prevDate]
#             return_self_per = round(return_self, 4) * 100
#             return_sp500_per = round(return_sp500, 4) * 100

        
        return True, return_self_per - return_sp500_per # relative return
    except Exception as e:
        return False, 0

def main():
    raw_price_file = '../inputs/stockPrices_raw.json'
    with open(raw_price_file) as file:
        print("Loading price info ...")
        priceSet = json.load(file)
        dateSet = priceSet['AAPL']['adjClose'].keys()

    returns = {'short': {}, 'mid': {}, 'long': {}} # 1-depth dictionary
    for num, ticker in enumerate(priceSet):
        print(num, ticker)
        for term in ['short', 'mid', 'long']:
            returns[term][ticker] = {} # 2-depth dictionary
        for day in dateSet:
            date = datetime.datetime.strptime(day, "%Y-%m-%d").strftime("%Y%m%d") # change date 2014-01-01 to 20140101
            tag_short, return_short = calc_mid_long_return(ticker, day, 1, priceSet)
            tag_mid, return_mid = calc_mid_long_return(ticker, day, 7, priceSet)
            tag_long, return_long = calc_mid_long_return(ticker, day, 28, priceSet)
            if tag_short:
                returns['short'][ticker][date] = return_short
            if tag_mid:
                returns['mid'][ticker][date] = return_mid
            if tag_long:
                returns['long'][ticker][date] = return_long

    with open('../inputs/stockReturns.json', 'w') as outfile:
        json.dump(returns, outfile, indent=4)


In [4]:
if __name__ == "__main__":
    main()

Loading price info ...
0 ^GSPC
1 AA
2 AAAP
3 A
4 AAL
5 AAN
6 AAPL
7 AB
8 ABAX
9 ABB
10 ABBV
11 ABCO
12 ABEO
13 ABMD
14 ABR
15 ABRN
16 ABT
17 ABX
18 ACAD
19 ACAT
20 ACC
21 ACFC
22 ACGL
23 ACGLP
24 ACLS
25 ACN
26 ACOR
27 ACRS
28 ACRX
29 ACST
30 ACU
31 ACUR
32 ADBE
33 ADHD
34 ADK
35 ADMA
36 ADM
37 ADMS
38 ADP
39 ADPT
40 ADS
41 ADX
42 ADXS
43 AEB
44 AED
45 AEG
46 AEGN
47 AEH
48 AEHR
49 AEK
50 AEM
51 AEO
52 AEP
53 AER
54 AERI
55 AET
56 AEUA
57 AEY
58 AEZS
59 AFA
60 AFAM
61 AF
62 AFC
63 AFGE
64 AFGH
65 AFMD
66 AFSD
67 AFW
68 AGCO
69 AGFS
70 AGI
71 AGIIL
72 AGIO
73 AGN
74 AGNCP
75 AGO
76 AGR
77 AGTC
78 AGU
79 AHC
80 AHL
81 AHP
82 AIA
83 AI
84 AIB
85 AIC
86 AIG
87 AIMT
88 AINC
89 AIR
90 AIRM
91 AIRT
92 AIW
93 AIY
94 AJG
95 AKAM
96 AKAO
97 AKBA
98 AKER
99 AKS
100 ALB
101 ALBO
102 ALCO
103 ALDX
104 ALE
105 ALGT
106 ALIM
107 ALJ
108 ALJJ
109 ALK
110 ALKS
111 ALL
112 ALLT
113 ALLY
114 ALNY
115 ALOT
116 ALR
117 ALV
118 ALXN
119 AMAG
120 AMAT
121 AMBR
122 AMC
123 AMCC
124 AMCX
125 AMDA
126 AMG
127 A

970 HK
971 HLF
972 HLG
973 HLIT
974 HLS
975 HLT
976 HMC
977 HMNF
978 HMNY
979 HNNA
980 HNR
981 HOG
982 HOLX
983 HON
984 HOS
985 HPE
986 HPJ
987 HPP
988 HPQ
989 HPT
990 HRB
991 HRI
992 HRL
993 HRS
994 HRTX
995 HRZN
996 HSBC
997 HSC
998 HSEA
999 HSEB
1000 HSGX
1001 HSON
1002 HSY
1003 HTBX
1004 HTF
1005 HTGX
1006 HT
1007 HTM
1008 HUM
1009 HUN
1010 HURN
1011 HW
1012 HZO
1013 IAC
1014 IART
1015 IBIO
1016 IBKCO
1017 IBKCP
1018 IBM
1019 IBN
1020 ICBK
1021 ICCC
1022 ICE
1023 ICLR
1024 ICPT
1025 IDCC
1026 IDN
1027 IDXG
1028 IEC
1029 IEP
1030 IFF
1031 IGC
1032 IGT
1033 IHC
1034 IHG
1035 IIJI
1036 IIN
1037 IKGH
1038 ILMN
1039 IMAX
1040 IMGN
1041 IMH
1042 IMI
1043 IMKTA
1044 IMMR
1045 IMMU
1046 IMMY
1047 IMN
1048 IMNP
1049 IMO
1050 IMOS
1051 INAP
1052 INCY
1053 INFY
1054 INGN
1055 INO
1056 INOV
1057 INSE
1058 INSG
1059 INSM
1060 INSY
1061 INTC
1062 INTL
1063 INTU
1064 INTX
1065 INUV
1066 INVE
1067 INVN
1068 INVT
1069 IOC
1070 IO
1071 IONS
1072 IOTS
1073 IPCI
1074 IPDN
1075 IPG
1076 IPHS
1077 IPXL


1831 SKIS
1832 SKLN
1833 SKX
1834 SKY
1835 SKYS
1836 SKYW
1837 SLB
1838 SLF
1839 SLMAP
1840 SLMBP
1841 SLM
1842 SLTB
1843 SLW
1844 SMBC
1845 SMBK
1846 SMED
1847 SMFG
1848 SMG
1849 SMMF
1850 SMRT
1851 SMSI
1852 SM
1853 SMTC
1854 SMTX
1855 SNAK
1856 SNDE
1857 SNDX
1858 SNE
1859 SNN
1860 SNOA
1861 SNOW
1862 SNP
1863 SNPS
1864 SN
1865 SNSS
1866 SNX
1867 SNY
1868 SOFO
1869 SOHOB
1870 SON
1871 SONS
1872 SPAN
1873 SPAR
1874 SPB
1875 SPGI
1876 SPG
1877 SPIL
1878 SPI
1879 SPKE
1880 SPLP
1881 SPLS
1882 SPNE
1883 SPN
1884 SPP
1885 SPR
1886 SPRT
1887 SPWR
1888 SQBG
1889 SQM
1890 SQNS
1891 SQ
1892 SRAX
1893 SRCLP
1894 SRE
1895 SRPT
1896 SRRA
1897 SRTS
1898 SSBI
1899 SSH
1900 SSL
1901 S
1902 SSRI
1903 SSY
1904 STAF
1905 STAR
1906 STBA
1907 STE
1908 STKL
1909 STLD
1910 STLR
1911 STLRU
1912 STL
1913 STLY
1914 STML
1915 STM
1916 STON
1917 STO
1918 STRL
1919 STRT
1920 STT
1921 STX
1922 STZ.B
1923 STZ
1924 SUM
1925 SUP
1926 SU
1927 SVBI
1928 SVU
1929 SWJ
1930 SWKS
1931 SWK
1932 SWN
1933 SXC
1934 SXL
1935